<a href="https://colab.research.google.com/github/fheydari0024/Dynamic-Time-Warping-exercise-/blob/main/notebooks/DTW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade numba

In [ ]:
!pip install librosa

In [ ]:
import librosa
import numpy as np

file_bale = '/content/drive/MyDrive/DTW/Bale.mp3'
file_kheyr = '/content/drive/MyDrive/DTW/Kheyr.mp3'
y_bale, sr_bale = librosa.load(file_bale, sr=None)
y_kheyr, sr_kheyr = librosa.load(file_kheyr, sr=None)

# تقسیم سیگنال‌ها به فریم‌ها
frame_length = 300
hop_length = 100
frames_bale = librosa.util.frame(y_bale, frame_length=frame_length, hop_length=hop_length)
frames_kheyr = librosa.util.frame(y_kheyr, frame_length=frame_length, hop_length=hop_length)

print(f"Length of the 'bale' signal: {len(y_bale)} samples")
print(f"Length of the 'kheyr' signal: {len(y_kheyr)} samples")

print(f"Number of frames for the word 'bale': {frames_bale.shape[1]}")
print(f"Number of frames for the word 'kheyr': {frames_kheyr.shape[1]}")

<ipython-input-32-edb2b3160720>:6: UserWarning: PySoundFile failed. Trying audioread instead.
  y_bale, sr_bale = librosa.load(file_bale, sr=None)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
<ipython-input-32-edb2b3160720>:7: UserWarning: PySoundFile failed. Trying audioread instead.
  y_kheyr, sr_kheyr = librosa.load(file_kheyr, sr=None)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Length of the 'bale' signal: 68608 samples
Length of the 'kheyr' signal: 82944 samples
Number of frames for the word 'bale': 684
Number of frames for the word 'kheyr': 827


In [ ]:
def extract_mfcc(y, sr, frame_length=300, hop_length=100, n_mfcc=16):
    frames = split_into_frames(y, frame_length, hop_length)

    mfcc_features = []
    for i in range(frames.shape[1]):
        frame = frames[:, i]
        mfcc = librosa.feature.mfcc(y=frame, sr=sr, n_mfcc=n_mfcc)
        mfcc_features.append(mfcc.T.flatten())

    mfcc_features = np.array(mfcc_features)
    return mfcc_features

# استخراج ویژگی‌های MFCC هر کلمه
mfcc_bale = extract_mfcc(y_bale, sr_bale, frame_length=300, hop_length=100, n_mfcc=16)
mfcc_kheyr = extract_mfcc(y_kheyr, sr_kheyr, frame_length=300, hop_length=100, n_mfcc=16)

# نمایش ابعاد ویژگی‌های MFCC
print(f"MFCC feature dimensions for 'bale': {mfcc_bale.shape}")
print(f"MFCC feature dimensions for 'kheyr': {mfcc_kheyr.shape}")

/usr/local/lib/python3.11/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=300
  warnings.warn(


MFCC feature dimensions for 'bale': (684, 16)
MFCC feature dimensions for 'kheyr': (827, 16)


In [ ]:
from scipy.spatial.distance import cdist
file_test = '/content/drive/MyDrive/DTW/Test.mp3'
y_test, sr_test = load_audio(file_test)
mfcc_test = extract_mfcc(y_test, sr_test, frame_length=300, hop_length=100, n_mfcc=16)

# محاسبه فاصله DTW
def calculate_dtw(mfcc1, mfcc2):
    distance_matrix = cdist(mfcc1, mfcc2, metric='euclidean')
    return dynamic_time_warping(distance_matrix)

def dynamic_time_warping(distance_matrix):
    n, m = distance_matrix.shape
    dtw = np.zeros((n+1, m+1))
    dtw[0, 1:] = np.inf
    dtw[1:, 0] = np.inf

    for i in range(1, n+1):
        for j in range(1, m+1):
            cost = distance_matrix[i-1, j-1]
            dtw[i, j] = cost + min(dtw[i-1, j], dtw[i, j-1], dtw[i-1, j-1])
    return dtw[n, m]

dtw_bale = calculate_dtw(mfcc_test, mfcc_bale)
dtw_kheyr = calculate_dtw(mfcc_test, mfcc_kheyr)

print(f"DTW distance between the test word and 'bale': {dtw_bale}")
print(f"DTW distance between the test word and 'kheyr': {dtw_kheyr}")

if dtw_bale < dtw_kheyr:
    print("The word test is more like 'yes'.")
else:
    print("The word test is more like 'kheyr'.")

<ipython-input-30-025f11c664f8>:10: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, sr=None)  # بارگذاری فایل صوتی
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/usr/local/lib/python3.11/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=300
  warnings.warn(


DTW distance between the test word and 'bale': 47106.84067470656
DTW distance between the test word and 'kheyr': 47967.581602629
The word test is more like 'yes'.
